In [51]:
import argparse
import os
import glob

import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import random
import open3d as o3d
from matplotlib import cm
from plyfile import PlyData, PlyElement
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box


from sphere import Sphere
from dh_grid import DHGrid


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
def loadPointCloudFromPath(path_to_point_cloud):
        # print(f'Loading point cloud from {path_to_point_cloud}')
        plydata = PlyData.read(path_to_point_cloud)
        vertex = plydata['vertex']
        x = vertex['x']
        y = vertex['y']
        z = vertex['z']
        if 'scalar' in vertex._property_lookup:
            i = vertex['scalar']
        elif 'intensity' in vertex._property_lookup:
            i = vertex['intensity']
        else:
            i = plydata['vertex'][plydata.elements[0].properties[3].name]

        s = np.zeros(x.shape[0])
        return np.concatenate((x, y, z, i, s), axis=0).reshape(5, len(x)).transpose()

def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])    
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
        


In [62]:


# export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/raw'
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/os0'

all_files = sorted(glob.glob(export_ds + '/*.ply'))

n_scans = 200
all_sem_clouds = []
for i in range(0, n_scans):    
    all_sem_clouds.append(loadPointCloudFromPath(all_files[i]))

print(f'Loaded {len(all_sem_clouds)} clouds from disk.')
print(f'Shape of one cloud is {all_sem_clouds[0].shape}')
visualize_pointcloud(loadPointCloudFromPath(all_files[0]))

Loaded 200 clouds from disk.
Shape of one cloud is (131072, 5)


In [21]:
for i in range(0, n_scans):
    filename = f"{export_ds}/processed/scan{i}.npy"
    np.save(filename, all_sem_clouds[i])

In [63]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
    sample_sphere = Sphere(all_sem_clouds[sample_idx])    
    features = sample_sphere.sampleUsingGrid(grid)
    return features

In [64]:
bw = 100
grid, _ = DHGrid.CreateGrid(bw)
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
sem_features = process_map(sample_func, sem_idx, max_workers=10, chunksize=25)            
print(f"Processing done.")

  0%|          | 0/200 [00:00<?, ?it/s]

Processing done.


In [65]:
filename = f"{export_ds}/features/os0.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

Wrote features to /media/berlukas/Data2/datasets/nuscenes/processed/os0/features/os0.npy.


# Visualize
----------------------

In [54]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)


def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd   

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def convert_sphere(sph, feature_idx, n_features, bw, trans = 0.0):
    sph = np.reshape(sph, (n_features, -1)).T
    
    pc = SamplingPointCloud.from_bw(bw, 1)
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    return np.column_stack((points_xyz, sph[:,feature_idx]))
    
def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)
    o3d.visualization.draw_geometries([pcd])

def visualize_sphere(sph, feature_idx = 0, n_features = 3, bw = 100):
    points_xyzi = convert_sphere(sph, feature_idx, n_features, bw)
    visualize_pointcloud(points_xyzi)

In [57]:
cur_sem_cloud = sem_features[0]
cur_sem_cloud = np.reshape(cur_sem_cloud, (2, -1)).T

pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
print(f"sampling pointcloud shape is {points_xyz.shape}")
print(f"feature shape is {cur_sem_cloud.shape}")

visualize_sphere(sem_features[0], 1, 2, 100)

sampling pointcloud shape is (40000, 3)
feature shape is (40000, 2)
